In [2]:

# from groundingdino.util.inference import load_model, load_image, predict, annotate
# import cv2

# model = load_model("groundingdino/config/GroundingDINO_SwinT_OGC.py", "/home/pearson/Projects/ECE253/GroundingDINO/weights/groundingdino_swint_ogc.pth")
# IMAGE_PATH = "/home/pearson/Projects/ECE253/Image_process/example_jpg/output_CLAHE/output_clahe_IMG_3580.jpeg"
# TEXT_PROMPT = "person . car ."
# BOX_TRESHOLD = 0.35
# TEXT_TRESHOLD = 0.25

# image_source, image = load_image(IMAGE_PATH)

# boxes, logits, phrases = predict(
#     model=model,
#     image=image,
#     caption=TEXT_PROMPT,
#     box_threshold=BOX_TRESHOLD,
#     text_threshold=TEXT_TRESHOLD,
#     device="cpu"
# )

# annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
# cv2.imwrite("output_clahe_IMG_3580_annoted.jpg", annotated_frame)

# num_boxes = len(boxes)

# if num_boxes > 0:
#     avg_score = logits.mean().item()
    
#     print(f"Number of detected boxes: {num_boxes}")
#     print(f"Average Confidence (Average Score): {avg_score:.4f}")
    
#     print(f"Highest Score: {logits.max().item():.4f}")
#     print(f"Lowest Score: {logits.min().item():.4f}")
# else:
#     print("No target detected.")

In [ ]:
import os
import cv2
import torch
import glob
import numpy as np  
from tqdm import tqdm
from groundingdino.util.inference import load_model, load_image, predict, annotate

def process_single_folder(model, input_dir, output_dir, config):

    text_prompt = config["text_prompt"]
    box_threshold = config["box_threshold"]
    text_threshold = config["text_threshold"]
    device = config["device"]

    os.makedirs(output_dir, exist_ok=True)

    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.PNG', '*.JPG', '*.JPEG', '*.BMP']
    image_paths = []
    for ext in image_extensions:
        image_paths.extend(glob.glob(os.path.join(input_dir, ext)))
    
    print(f"\n{'-'*60}")
    print(f"Processing Folder: {input_dir}")
    print(f"Output Directory:  {output_dir}")
    print(f"Images Found:      {len(image_paths)}")
    print(f"{'-'*60}")

    if not image_paths:
        print("No images found. Skipping.")
        return


    all_scores_in_folder = []

    per_image_stats = []

    folder_total_boxes = 0


    for img_path in tqdm(image_paths, desc="Progress"):
        filename = os.path.basename(img_path)
        
        try:
            image_source, image = load_image(img_path)

            boxes, logits, phrases = predict(
                model=model,
                image=image,
                caption=text_prompt,
                box_threshold=box_threshold,
                text_threshold=text_threshold,
                device=device
            )


            annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
            cv2.imwrite(os.path.join(output_dir, f"annotated_{filename}"), annotated_frame)


            current_count = len(boxes)

            current_scores = logits.tolist() 

            per_image_stats.append({
                "filename": filename,
                "box_count": current_count,
                "scores": current_scores
            })


            folder_total_boxes += current_count
            all_scores_in_folder.extend(current_scores)

        except Exception as e:
            print(f"Error processing {filename}: {e}")


    print(f"\n>>> STATS REPORT FOR FOLDER: {os.path.basename(input_dir)}")
    

    print(f"Total Boxes Detected: {folder_total_boxes}")

    if len(all_scores_in_folder) > 0:

        folder_mean = np.mean(all_scores_in_folder)
        folder_variance = np.var(all_scores_in_folder)
        
        print(f"Average Confidence:   {folder_mean:.4f}")
        print(f"Confidence Variance:  {folder_variance:.4f}")
    else:
        print("Average Confidence:   N/A (No detections)")
        print("Confidence Variance:  N/A")
    

    # for stat in per_image_stats:
    #     print(f"  File: {stat['filename']}, Count: {stat['box_count']}, Scores: {stat['scores']}")

    print(f"{'='*60}\n")


def main():

    cfg = {
        "config_path": "groundingdino/config/GroundingDINO_SwinT_OGC.py",
        "weights_path": "/home/pearson/Projects/ECE253/GroundingDINO/weights/groundingdino_swint_ogc.pth",
        "text_prompt": "person . car .",
        "box_threshold": 0.35,
        "text_threshold": 0.25,
        "device": "cpu"
    }

    folders_to_process = [
        (
            "../dataset", 
            "../results/annoted/original/"
        ),
        (
            "../results/CLAHE/", 
            "../results/annoted/CLAHE/"
        ),
        (
            "../results/CLAHE_Wiener/", 
            "../results/annoted/CLAHE_Wiener/"
        ),
        (
            "../results/CLAHE_NAFNet/", 
            "../results/annoted/CLAHE_NAFNet/"
        ),
        (
            "../results/ZeroDCE/", 
            "../results/annoted/ZeroDCE/"
        ),
        (
            "../results/ZeroDCE_Wiener/", 
            "../results/annoted/ZeroDCE_Wiener/"
        ),
        (
            "../results/ZeroDCE_NAFNet/", 
            "../results/annoted/ZeroDCE_NAFNet/"
        ),
    ]


    print(f"Loading Model from {cfg['weights_path']} ...")
    model = load_model(cfg["config_path"], cfg["weights_path"])


    for input_dir, output_dir in folders_to_process:
        process_single_folder(model, input_dir, output_dir, cfg)

    print("All batch processing complete.")

if __name__ == "__main__":
    main()

Loading Model from /home/pearson/Projects/ECE253/GroundingDINO/weights/groundingdino_swint_ogc.pth ...
final text_encoder_type: bert-base-uncased



------------------------------------------------------------
Processing Folder: ../dataset
Output Directory:  ../results/annoted/original/
Images Found:      169
------------------------------------------------------------


Progress: 100%|██████████| 169/169 [07:10<00:00,  2.55s/it]



>>> STATS REPORT FOR FOLDER: dataset
Total Boxes Detected: 400
Average Confidence:   0.5617
Confidence Variance:  0.0239


------------------------------------------------------------
Processing Folder: ../results/CLAHE/
Output Directory:  ../results/annoted/CLAHE/
Images Found:      169
------------------------------------------------------------


Progress: 100%|██████████| 169/169 [07:17<00:00,  2.59s/it]


...

Progress: 100%|██████████| 169/169 [07:01<00:00,  2.49s/it]



>>> STATS REPORT FOR FOLDER: 
Total Boxes Detected: 321
Average Confidence:   0.5427
Confidence Variance:  0.0227


------------------------------------------------------------
Processing Folder: ../results/ZeroDCE_NAFNet/
Output Directory:  ../results/annoted/ZeroDCE_NAFNet/
Images Found:      169
------------------------------------------------------------


Progress: 100%|██████████| 169/169 [06:56<00:00,  2.46s/it]


>>> STATS REPORT FOR FOLDER: 
Total Boxes Detected: 549
Average Confidence:   0.5380
Confidence Variance:  0.0219

All batch processing complete.


: 